In [1]:
from netgen.geom2d import SplineGeometry
from ngsolve import *
from ngsolve.internal import *
from xfem import *
from xfem.lsetcurv import *
from math import pi,e
from numpy import linspace

ngsglobals.msg_level = 2
DrawDC = MakeDiscontinuousDraw(Draw)

interactive = 1

importing ngsxfem-2.1.2504


In [2]:
# physical parameters for Biot
mu  = 1e6
lam = 1e6
# alpha = 1
alpha = 1
K = 1e2 # k^-1
nu = 1
# s0 = 10
s0 = 1e-2

def Stress(strain):
    return 2*mu*strain + lam*Trace(strain)*Id(2)

In [3]:
# DG space order
order_eta = 2
order_u = 2
order_p = 1
# penalty parameters
beta_eta = 200
beta_u = 200
# ghost penalty parameters
gamma_s = 0.1
gamma_u = 0.1
gamma_p = 0.1
gamma_m = 0
tau_p = 0

In [23]:
eta_x = 0
eta_y = 0
u_x = 0
u_y = IfPos(y-0.5,10*x*(x-1),-10*x*(x-1))
exact_eta = CF((eta_x, eta_y))
exact_u = CF((u_x,u_y))

# 向量形式
fe = CF((0, 0))
fm = CF((0, 0))
fp = 0
etaD = exact_eta
uD = exact_u

In [24]:
# Construct background mesh
# Quadrilateral (or simplicial mesh)
quad_mesh = False
# Mesh diameter
# h0 = 1/32
h0 = 0.01
# Geometry and Mesh
square = SplineGeometry()
square.AddRectangle((0, 0), (1, 1), bcs=('b','r','t','l'))
ngmesh = square.GenerateMesh(maxh=h0, quad_dominated=quad_mesh)
mesh = Mesh(ngmesh)
# Draw(mesh)

 Generate Mesh from spline geometry


In [25]:
circle1 = 9/2500-(x-0.25)**2-(y-0.7)**2
circle2 = 0.01-(x-0.15)**2-(y-0.45)**2
a = 0.12
b = 0.06
circle3 = 1 - (x-y+0.2)**2/(2*a**2) - (x+y-1.34)**2/(2*b**2)

In [26]:
# 参数设置
x0, y0 = 0.7, 0.5  # 中心位置
xc = 0.1           # 焦点间距的一半，决定花生总长度
a = 0.11         # 稍微大于 xc，决定凹陷深度
theta = pi/3  # 旋转角度

# 1. 坐标变换
dx = x - x0
dy = y - y0
xr = dx * cos(theta) + dy * sin(theta)
yr = -dx * sin(theta) + dy * cos(theta)

# 2. 计算到旋转后焦点的距离平方
# 焦点在局部坐标系的 (xc, 0) 和 (-xc, 0)
d1_sq = (xr - xc)**2 + yr**2
d2_sq = (xr + xc)**2 + yr**2

# 3. 构造 Level Set (内部为正，外部为负)
# 使用 a^2 - sqrt(...) 保证量级为 L^2
circle4 = a**2 - sqrt(d1_sq * d2_sq)

In [27]:
circle5 = 0.01-(x-0.35)**2-(y-0.25)**2
circle6 = 4/625-(x-0.85)**2-(y-0.85)**2
a = 0.2
b = 0.05
circle7 = 1 - (x-0.7)**2/(a**2) - (y-0.15)**2/(b**2)

In [28]:

# circle4 = 0.06**4 - ((x-0.6)**2+(y-0.35)**2)*((x-0.5)**2+(y-0.35)**2) 
# dx1 = (x-0.55)-0.1; dx2 = (x-0.55)+0.1; dy = (y-0.35)
# circle4 = 0.11**4 - (dx1**2 + dy**2)*(dx2**2 + dy**2)
f1 = IfPos(circle1-circle2,circle1,circle2)
f2 = IfPos(f1-circle3,f1,circle3)
f3 = IfPos(f2-circle4, f2, circle4)
f4 = IfPos(f3-circle5, f3, circle5)
f5 = IfPos(f4-circle6, f4, circle6)
levelset = IfPos(f5-circle7, f5, circle7).Compile()

lsetmeshadap = LevelSetMeshAdaptation(mesh, order=1, threshold=0.1,
                                      discontinuous_qn=True)
deformation = lsetmeshadap.CalcDeformation(levelset)
lsetp1 = lsetmeshadap.lset_p1

In [29]:
# Element, facet and dof marking w.r.t. boundary approximation with lsetp1:
ci = CutInfo(mesh, lsetp1)
hasneg = ci.GetElementsOfType(HASNEG)
hasif = ci.GetElementsOfType(IF)
Draw(BitArrayCF(hasneg),mesh)
# Draw(BitArrayCF(hasif),mesh)

# facets used for stabilization:
ba_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasif)
ba_surround_facets = GetElementsWithNeighborFacets(mesh,ba_facets)
interior_facets = GetFacetsWithNeighborTypes(mesh, a=hasneg, b=hasneg)
in_surround_facets = GetElementsWithNeighborFacets(mesh,interior_facets)
# Draw(BitArrayCF(ba_surround_facets), mesh, "surrounding_facets") 
# Draw(BitArrayCF(in_surround_facets), mesh, "surrounding_facets") 


WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

In [30]:
kappaminus = CutRatioGF(ci)
kappaminus_values = kappaminus.vec.FV().NumPy()

# 找出非零值中的最小值
# 这里需要过滤掉那些没有被切割的单元的值（通常是0）
# min_value = 1.0 # 初始化一个比任何比率都大的值
# for value in kappaminus_values:
#     if value > 0 and value < min_value:
#         min_value = value

# print(f"所有被切割单元中，比值的最小值为: {min_value}")

# Or
positive_values = [v for v in kappaminus_values if v > 0]
if positive_values:
    min_value_pythonic = min(positive_values)
    print(f"所有被切割单元中，比值的最小值为 (Pythonic): {min_value_pythonic}")
else:
    print("没有找到被切割的单元。")

所有被切割单元中，比值的最小值为 (Pythonic): 6.222050818628951e-07


In [31]:
#Unfitted DG spaces
Ehbase = VectorL2(mesh, order=order_eta, dirichlet=[], dgjumps=True) # space for displacement
Uhbase = VectorL2(mesh, order=order_u, dirichlet=[], dgjumps=True) # space for velocity
# Phbase = L2(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
Phbase = H1(mesh, order=order_p, dirichlet=[], dgjumps=True) # space for pressure 
E = Compress(Ehbase, GetDofsOfElements(Ehbase, ci.GetElementsOfType(HASNEG)))
U = Compress(Uhbase, GetDofsOfElements(Uhbase, ci.GetElementsOfType(HASNEG)))
P = Compress(Phbase, GetDofsOfElements(Phbase, ci.GetElementsOfType(HASNEG)))
fes = E*U*P
(eta,u,p), (kxi,v,q) = fes.TnT()

In [32]:
# Define the jumps and the averages
# n = Normalize(grad(lsetp1)) # outer normal vector on the boundary
n = 1.0 / Norm(grad(lsetp1)) * grad(lsetp1)
ne = specialcf.normal(2) # normal vectors on faces
h = specialcf.mesh_size  

strain_eta = Sym(Grad(eta))
strain_kxi = Sym(Grad(kxi))
mean_stress_eta = 0.5*(Stress(Sym(Grad(eta)))+Stress(Sym(Grad(eta.Other()))))*ne
mean_stress_kxi = 0.5*(Stress(Sym(Grad(kxi)))+Stress(Sym(Grad(kxi.Other()))))*ne
jump_eta = eta - eta.Other()
jump_kxi = kxi - kxi.Other()


jump_p = p - p.Other()
jump_q = q - q.Other()
mean_p = 0.5*(p + p.Other())
mean_q = 0.5*(q + q.Other())


mean_dudn = 0.5*(Grad(u)+Grad(u.Other()))*ne
mean_dvdn = 0.5*(Grad(v)+Grad(v.Other()))*ne
jump_u = u - u.Other()
jump_v = v - v.Other()
 

In [33]:
# integration domains:
domega = dCut(lsetp1, NEG, definedonelements=hasneg, deformation=deformation)
dk = dCut(lsetp1, NEG, skeleton=True, definedonelements=interior_facets,
          deformation=deformation)
dsc = dCut(lsetp1, IF, definedonelements=hasif, deformation=deformation) # 在界面上的积分
dw = dFacetPatch(definedonelements=ba_facets, deformation=deformation)
dsbar = ds(skeleton=True,definedon=mesh.Boundaries('t|b|l|r')) # 在外边界上的积分
dseta = ds(skeleton=True)

In [34]:
Ah = BilinearForm(fes)
# Ae
Ah += 2*mu*InnerProduct(strain_eta,strain_kxi)*domega + lam*div(eta)*div(kxi)*domega \
        - (InnerProduct(mean_stress_eta,jump_kxi) + InnerProduct(mean_stress_kxi,jump_eta))*dk \
        - (InnerProduct(Stress(Sym(Grad(eta)))*ne,kxi) + InnerProduct(Stress(Sym(Grad(kxi)))*ne,eta))*dseta

Ah +=  beta_eta/h*(2*mu*InnerProduct(jump_eta,jump_kxi)+lam*InnerProduct(jump_eta,ne)*InnerProduct(jump_kxi,ne)) * dk
Ah += beta_eta/h*(2*mu*InnerProduct(eta,kxi) + lam*InnerProduct(eta,ne)*InnerProduct(kxi,ne))*dseta


# order=1 i_s 
Ah += gamma_s / (h**2) * (eta - eta.Other()) * (kxi - kxi.Other()) * dw

# Be
Ah += -alpha*(div(kxi)*p*domega - mean_p*jump_kxi*ne*dk - p*kxi*ne*dseta)

              
# Am
Ah += nu*InnerProduct(Grad(u),Grad(v))*domega \
        - nu*(InnerProduct(mean_dudn,jump_v) + InnerProduct(mean_dvdn,jump_u) - beta_u/h*InnerProduct(jump_u,jump_v))*dk \
        - nu*(InnerProduct(Grad(u)*ne,v) + InnerProduct(Grad(v)*ne,u) - beta_u/h*InnerProduct(u,v))*dsbar\
        + K*InnerProduct(u,v)*domega


# ghost penalty for velocity
Ah += gamma_u / (h**2) * (u - u.Other()) * (v - v.Other()) * dw



 # Bm 
Ah += -div(v)*p*domega + mean_p*jump_v*ne*dk + p*v*ne*dsbar

# -Bm
Ah += div(u)*q*domega - mean_q*jump_u*ne*dk - q*u*ne*dsbar

# order=1 i_p 
# Ah += gamma_p * (h**3) * (grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dk
Ah += gamma_p * jump_p * jump_q * dw

# M
Ah += s0*p*q*domega + gamma_m*(h**3)*(grad(p)*ne - grad(p.Other())*ne)*(grad(q)*ne - grad(q.Other())*ne) * dw

# -Be
Ah += alpha*(div(eta)*q*domega - mean_q*jump_eta*ne*dk - q*eta*ne*dseta)

# stabilization
Ah += tau_p*h*jump_p*jump_q*dk
Ah.Assemble()

In [35]:
# r.h.s
lh = LinearForm(fes) 
lh += fe*kxi*domega - InnerProduct(etaD,Stress(Sym(Grad(kxi)))*ne)*dseta 
lh += beta_eta/h*(2*mu*InnerProduct(etaD,kxi) + lam*InnerProduct(etaD,ne)*InnerProduct(kxi,ne))*dseta
lh += fm*v*domega 
lh += - (nu*InnerProduct(uD,Grad(v)*ne) - nu*beta_u/h*uD*v)*dsbar
lh += fp*q*domega 
lh += - alpha*q*etaD*ne*dseta - q*uD*ne*dsbar
lh.Assemble()

In [36]:
# help(CGSolver)
# help(Preconditioner)

In [37]:
gfu = GridFunction(fes)
gfu.vec.data = Ah.mat.Inverse() * lh.vec
# pre = Preconditioner(Ah, "bddc")
# solvers.MinRes(mat=Ah.mat,
#                rhs=lh.vec,
#                pre=pre,
#                sol=gfu.vec,
#                printrates=True,
#                initialize=False)

# solvers.MinRes (mat=K, pre=C, rhs=rhs, sol=sol, printrates=True, initialize=False);
# gfu.vec.data = inv * lh.vec

In [38]:
# import numpy as np
# import scipy.sparse as sp
# import scipy.io as sio

# # NGSolve matrix -> SciPy CSR
# A_csr = Ah.mat.CSR()

# # CSR components
# A_data = A_csr[0]
# A_indices = A_csr[1]
# A_indptr = A_csr[2]
# A_shape = (Ah.mat.height, Ah.mat.width)

# A = sp.csr_matrix((A_data, A_indices, A_indptr), shape=A_shape)
# b = lh.vec.FV().NumPy().copy()
# sio.savemat("/mnt/d/ngsExperiments/Biot_Brinkman/Ex_riverbed/biot_brinkman.mat", {
#     "A": A,
#     "b": b,
#     # "x": x,   # 可选
# })


In [39]:
# error_eta = sqrt(Integrate((gfu.components[0] - exact_eta)**2* domega, mesh))
# error_u = sqrt(Integrate((gfu.components[1] - exact_u)**2 * domega, mesh))
# error_p = sqrt(Integrate((gfu.components[2] - exact_p)**2 * domega, mesh))

# print(error_eta)
# print(error_u)
# print(error_p)

In [40]:
# gff = GridFunction(fes)
# gff.components[0].Set(exact_eta)
# gff.components[1].Set(exact_u)

# grad_error_eta = Grad(gfu.components[0] - gff.components[0])
# grad_error_u = Grad(gfu.components[1] - gff.components[1])

# error_eta_H1 = sqrt(Integrate(InnerProduct(grad_error_eta,grad_error_eta)* domega, mesh))
# error_u_H1 = sqrt(Integrate(InnerProduct(grad_error_u,grad_error_u)*  domega, mesh))
# # error_p = sqrt(Integrate(grad(gfu.components[2] - exact_p)**2 * domega, mesh))

# print(error_eta_H1)
# print(error_u_H1)

In [41]:
mask = IfPos(levelset,0,1)
Draw(mask*gfu.components[0],mesh)
# Draw(mask*exact_eta,mesh)
# Draw(mask*(gfu.components[0]-exact_eta),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [42]:
Draw(mask*gfu.components[1],mesh)
# Draw(mask*exact_u,mesh)
# Draw(mask*(gfu.components[1]-exact_u),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
Draw (mask*gfu.components[1], mesh, vectors={"grid_size" : 1, "offset" : 0.5 } );

In [43]:
Draw(mask*gfu.components[2],mesh)
# Draw(mask*gfu.components[2],mesh,min=-1, max=1)
# Draw(mask*exact_p,mesh)

# Draw(mask*(gfu.components[2]-exact_p),mesh)

WebGuiWidget(layout=Layout(height='50vh', width='100%'), value={'gui_settings': {}, 'ngsolve_version': '6.2.25…

BaseWebGuiScene

In [ ]:
# import numpy as np
# import scipy.sparse as sp

# rows,cols,vals = Ah.mat.COO()
# A = sp.csr_matrix((vals,(rows,cols)))
# conds = np.linalg.cond(A.todense())

In [ ]:
# print(conds)

In [ ]:
vtk = VTKOutput(mesh,[gfu.components[0],gfu.components[1],gfu.components[2]],['eta','u','p'],"/mnt/d/ngs_output/Biot_Brinkman/Ex_riverbed3",subdivision=0)
vtk.Do()   #输出网格